In [2]:
from selenium import webdriver
import time
import requests
from bs4 import BeautifulSoup
import pprint
import pandas as pd
import datetime

In [3]:
driver = webdriver.Firefox()
driver.get("https://resultadosemponto.com/")

In [427]:
# openTable = driver.find_element_by_xpath("//h2[@class='entry-title']")
# openTable.click()

In [4]:
url = driver.current_url
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
titles = soup.find_all('h2', {'class':'entry-title'})
articles = soup.find_all('article')

In [5]:
articles[0].find_all('h2', {'class':'entry-title'})[0].text.split()

['Jogo', 'Do', 'Bicho', 'PT-Rio', '16:20hs', 'Quinta-Feira', '25/03/2021']

In [443]:
date = articles[0].find_all('h2', {'class':'entry-title'})[0].text.split()[-1]
hour = articles[0].find_all('h2', {'class':'entry-title'})[0].text.split()[-3]

In [704]:
def get_data(articles):
    dfs=[]
    
    for j in range(len(articles)):
        
        # METADATA
        name1 = articles[j].find_all('h2', {'class':'entry-title'})[0].text.split()[3]       
        date = articles[j].find_all('h2', {'class':'entry-title'})[0].text.split()[-1]
        hour = articles[j].find_all('h2', {'class':'entry-title'})[0].text.split()[-3]
        objDate = datetime.datetime.strptime( (date + " " + hour[:-2]), '%d/%m/%Y %H:%M' )        
        
        # DATA
        tds = articles[j].find_all('td')
        ns = ['1°','2°','3°','4°','5°','6°','7°']
        PREMIO = []
        RESULTADO = []
        GRUPO = []
        for i in range(len(tds)-3):
            n=i                                   
            for key in ns:                
                if key in tds[i]:
                    PREMIO.append(str(tds[n].text))
                    RESULTADO.append(int(tds[n+1].text))
                    GRUPO.append(int(tds[n+2].text))                                        
                elif len(tds) == 22:                    
                    m=0
                    while m < len(tds[:-1]):                    
                        PREMIO.append(str(tds[m].text))
                        RESULTADO.append(int(tds[m+1].text))
                        GRUPO.append(tds[m+2].text.split()[0])                        
                        m += 3
                    break
            n+=3
        df = pd.DataFrame({
            "DATA": objDate,
            "NOME": name1,
            "PREMIO": PREMIO,
            "RESULTADO": RESULTADO,
            "GRUPO": GRUPO
        }).drop_duplicates()
        df.to_csv('./jogodobixo_{}.csv'.format(j))
        dfs.append(df)

    return dfs

In [705]:
dfs = get_data(articles)

In [703]:
dfs[4

,DATA,NOME,PREMIO,RESULTADO,GRUPO
0,2021-03-24 19:20:00,Federal,1º,1936,09
1,2021-03-24 19:20:00,Federal,2º,2758,15
2,2021-03-24 19:20:00,Federal,3º,6920,05
3,2021-03-24 19:20:00,Federal,4º,9751,13
4,2021-03-24 19:20:00,Federal,5º,4908,02
5,2021-03-24 19:20:00,Federal,6º,6273,19
6,2021-03-24 19:20:00,Federal,7º,159,15


In [433]:
articles[i].find_all('h2', {'class':'entry-title'})[0].text.split()[-1]

IndexError: list index out of range

In [228]:
articles[i].find_all('h2', {'class':'entry-title'})[0].text.split()[3] 

'PT-Rio'

In [232]:
articles[i+3].find_all('h2', {'class':'entry-title'})[0].text.split()[-3]

'18:20hs'